### ENVIRONMENTAL RASTERS (SUMMER, 2025)

Workflow:
- Load manually downloaded Landsat rasters.
- Keep only necessary bands B2–B7 + QA_PIXEL for calculations.
- Compute NDVI, NDWI, NDBI.
- Perform cloud masking.
- Zonal stats over census tracts.
- Output final tract-level summer 2025 metrics.

For my own reference because this is a long and complicated notebook.

| INDEX                         | BAND              |
|---|---|
| Blue                          | B2                |
| Green                         | B3                |
| Red                           | B4                |
| NIR (Near Infrared)           | B5                |
| SWIR1 (Shortwave Infrared 1)  | B6                |
| SWIR2 (Shortwave Infrared)    | B7                |
| Pixel QA                      | QA_PIXEL          |
| ANG                           | Solar Geometry    |

In [ ]:
# Modules.
import os
from pathlib import Path
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import geopandas as gpd
from tqdm import tqdm
from rasterstats import zonal_stats

Scenes: 19
Usable scenes: 18


  0%|          | 0/18 [00:00<?, ?it/s]C:\Users\Tess\AppData\Local\Temp\ipykernel_138456\1432907060.py:161: RuntimeWarning: invalid value encountered in divide
  ndvi = (nir - red)   / (nir + red)
C:\Users\Tess\AppData\Local\Temp\ipykernel_138456\1432907060.py:162: RuntimeWarning: invalid value encountered in divide
  ndwi = (green - nir) / (green + nir)
C:\Users\Tess\AppData\Local\Temp\ipykernel_138456\1432907060.py:163: RuntimeWarning: invalid value encountered in divide
  ndbi = (swir1 - nir) / (swir1 + nir)
  6%|▌         | 1/18 [00:02<00:40,  2.39s/it]C:\Users\Tess\AppData\Local\Temp\ipykernel_138456\1432907060.py:161: RuntimeWarning: invalid value encountered in divide
  ndvi = (nir - red)   / (nir + red)
C:\Users\Tess\AppData\Local\Temp\ipykernel_138456\1432907060.py:162: RuntimeWarning: invalid value encountered in divide
  ndwi = (green - nir) / (green + nir)
C:\Users\Tess\AppData\Local\Temp\ipykernel_138456\1432907060.py:163: RuntimeWarning: invalid value encountered in divide

Saving composite rasters...
Done.
Tree canopy zonal stats...
Impervious zonal stats...
Land cover zonal stats...


C:\Users\Tess\AppData\Local\Temp\ipykernel_138456\1432907060.py:273: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  tracts.to_file(shape_out)
c:\Users\Tess\.conda\envs\geospatial\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'tree_canopy_frac' to 'tree_canop'
  ogr_write(
c:\Users\Tess\.conda\envs\geospatial\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'impervious_frac' to 'impervious'
  ogr_write(
c:\Users\Tess\.conda\envs\geospatial\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'landcover_mean' to 'landcover_'
  ogr_write(
c:\Users\Tess\.conda\envs\geospatial\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 197189855.796999991 of field shape_area of feature 1806 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\Tess\.conda\envs\geospatial

Saved: data\raster\shapes\nyc_tracts_env_2025.shp

---- NYC ENVIRONMENT SUMMARY ----
NYC Mean NDVI (JJA 2025): 0.1629
NYC Mean NDWI (JJA 2025): -0.1637


In [ ]:
# Paths.
landsat_dir = Path("data/raster/landsat")

nlcd_tree_path = Path("data/raster/nlcd_raster/nlcd_tree_canopy_2023.tiff")
nlcd_impervious_path = Path("data/raster/nlcd_raster/nyc_ncld_impervious_2024.tiff")
nlcd_landcov_path = Path("data/raster/nlcd_raster/nyc_ncld_land_cover_2024.tiff")

tracts_path = Path("data/nyc_tracts_2020/nyc_tracts_2020.shp")

output_dir = Path("data/raster/processed")
output_dir.mkdir(parents=True, exist_ok=True)

tracts = gpd.read_file(tracts_path).to_crs(4326)

# Dissolved NYC boundary to Shapely.
nyc_boundary = tracts.union_all()

In [ ]:
# Landsat grouping helper functions.
def extract_band_tag(file_path):
    """
    Extracts SR_B4 or QA_PIXEL from filename.
    Works for:
        LC08_..._SR_B4.TIF
        LC08_..._QA_PIXEL.TIF
    """
    name = file_path.name.replace(".TIF", "").replace(".tif", "")
    parts = name.split("_")

    if len(parts) >= 2:
        return "_".join(parts[-2:])
    
    return parts[-1]

def group_scenes(landsat_dir):
    """Groups files into scenes using first 7 underscore texts."""
    scenes = {}
    for file_path in landsat_dir.glob("*"):
        if file_path.is_dir():
            continue

        name = file_path.name

        if ("SR_B" not in name and "QA_PIXEL" not in name):
            continue

        tokens = name.split("_")
        if len(tokens) < 7:
            continue

        scene_id = "_".join(tokens[:7])
        scenes.setdefault(scene_id, []).append(file_path)

    return scenes

scene_dict = group_scenes(landsat_dir)
print("Scenes:", len(scene_dict))

# Required bands.
target_bands = {"SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7","QA_PIXEL"}

usable_scenes = []

for scene_id, files in scene_dict.items():
    tags = {extract_band_tag(f) for f in files}

    if target_bands.issubset(tags):
        usable_scenes.append(scene_id)

print("Usable scenes:", len(usable_scenes))
usable_scenes[:10]

In [ ]:
# Cloud masking helper function.
def cloud_mask(qa_array):
    """Return True for cloud-free pixels."""
    qa = qa_array.astype(np.uint16)
    
    cloud  = (1 << 3)
    shadow = (1 << 4)

    return ((qa & cloud) == 0) & ((qa & shadow) == 0)

# Load and crop to NYC helper function.
def load_and_crop(band_path, geom):
    """Crop a raster to NYC boundary, return array + updated profile."""
    with rasterio.open(band_path) as src:
        boundary_proj = gpd.GeoSeries([geom], crs = 4326).to_crs(src.crs).iloc[0]

        out_image, out_transform = mask(src, [boundary_proj], crop = True)

        profile = src.profile.copy()

        profile.update({
            "height": out_image.shape[1],
            "width":  out_image.shape[2],
            "transform": out_transform
        })

        return out_image[0].astype(np.float32), profile

In [ ]:
# Get summer Landsat index.
sum_ndvi = None
sum_ndwi = None
sum_ndbi = None
count = None

ref_profile = None
ref_shape = None

for scene_id in tqdm(usable_scenes):
    band_map = {extract_band_tag(f): f for f in scene_dict[scene_id]}

    # Load cropped bands.
    try:
        red, prof = load_and_crop(band_map["SR_B4"], nyc_boundary); current_profile = prof
        nir, = load_and_crop(band_map["SR_B5"], nyc_boundary)
        green, = load_and_crop(band_map["SR_B3"], nyc_boundary)
        blue, _ = load_and_crop(band_map["SR_B2"], nyc_boundary)
        swir1, _ = load_and_crop(band_map["SR_B6"], nyc_boundary)
        swir2, _ = load_and_crop(band_map["SR_B7"], nyc_boundary)
        qa, _ = load_and_crop(band_map["QA_PIXEL"], nyc_boundary)

    except Exception as e:
        print("Failed:", scene_id, e)

        continue

    # Get unclouded pixes.
    good_pixels = cloud_mask(qa)

    if good_pixels.sum() == 0:
        print("No valid pixels:", scene_id)
        continue

    # Apply mask.
    red = np.where(good_pixels, red, np.nan)
    nir = np.where(good_pixels, nir, np.nan)
    green = np.where(good_pixels, green, np.nan)
    blue = np.where(good_pixels, blue, np.nan)
    swir1 = np.where(good_pixels, swir1, np.nan)
    swir2 = np.where(good_pixels, swir2, np.nan)

    # Index.
    ndvi = (nir - red) / (nir + red)
    ndwi = (green - nir) / (green + nir)
    ndbi = (swir1 - nir) / (swir1 + nir)

    # Set the reference raster.
    if sum_ndvi is None:
        ref_profile = current_profile
        ref_shape = ndvi.shape

        sum_ndvi = np.zeros(ref_shape, dtype = np.float32)
        sum_ndwi = np.zeros(ref_shape, dtype = np.float32)
        sum_ndbi = np.zeros(ref_shape, dtype = np.float32)
        count = np.zeros(ref_shape, dtype = np.float32)

    # Align.
    def force_align(arr):
        if arr.shape == ref_shape:
            return arr

        destination = np.full(ref_shape, np.nan, dtype = np.float32)

        reproject(
            source = arr,
            destination = destination,
            src_transform = current_profile["transform"],
            src_crs = current_profile["crs"],
            dst_transform = ref_profile["transform"],
            dst_crs = ref_profile["crs"],
            src_nodata = np.nan,
            dst_nodata = np.nan,
            resampling = Resampling.bilinear,
        )

        return destination

    ndvi = force_align(ndvi)
    ndwi = force_align(ndwi)
    ndbi = force_align(ndbi)

    # Accumulate.
    valid = ~np.isnan(ndvi)
    sum_ndvi[valid] += ndvi[valid]
    sum_ndwi[valid] += ndwi[valid]
    sum_ndbi[valid] += ndbi[valid]
    count[valid] += 1

In [ ]:
# Error if there are no scenes.
if count is None or count.sum() == 0:
    raise RuntimeError("No scenes successfully processed.")

In [ ]:
# Final summer averages.
ndvi_final = sum_ndvi / count
ndwi_final = sum_ndwi / count
ndbi_final = sum_ndbi / count

In [ ]:
# Save rasters.
profile = ref_profile.copy()
profile.update(
    dtype = np.float32,
    count = 1,
    nodata = np.nan,
    compress = "lzw"
)

In [ ]:
# Save rasters with print checks.
def save_raster(arr, path):
    with rasterio.open(path, "w", **profile) as destination:
        destination.write(arr.astype(np.float32), 1)

print("Saving:")

save_raster(ndvi_final, output_dir / "ndvi_summer_2025.tif")
save_raster(ndwi_final, output_dir / "ndwi_summer_2025.tif")
save_raster(ndbi_final, output_dir / "ndbi_summer_2025.tif")

print("Done.")

In [ ]:
# Zonal statistics for NCLD.
def zonal_mean(rpath, gdf_or_geom):
    """Apply CRS zonal mean for tracts or city boundary."""
    with rasterio.open(rpath) as src:
        r_crs = src.crs

    if isinstance(gdf_or_geom, (gpd.GeoSeries, gpd.GeoDataFrame)):
        gdf = gdf_or_geom.to_crs(r_crs)
    else:
        gdf = gpd.GeoSeries([gdf_or_geom], crs = 4326).to_crs(r_crs)

    return zonal_stats(gdf, rpath, stats = ["mean"], nodata = np.nan)

# Print checks for the calculations.
print("Tree canopy zonal stats:")
tree = zonal_mean(nlcd_tree_path, tracts)
print("Impervious zonal stats:")
impervious = zonal_mean(nlcd_impervious_path, tracts)
print("Land cover zonal stats:")
landcover = zonal_mean(nlcd_landcov_path, tracts)

tracts["tree_canopy_frac"] = [t["mean"] for t in tree]
tracts["impervious_frac"] = [i["mean"] for i in impervious]
tracts["landcover_mean"] = [l["mean"] for l in landcover]

# Save as shapefile.
shape_out = output_dir.parent / "shapes" / "nyc_tracts_env_2025.shp"
shape_out.parent.mkdir(parents = True, exist_ok = True)

tracts.to_file(shape_out)

print("Saved:", shape_out)

In [ ]:
# Statistical summaries.
ndvi_path = output_dir / "ndvi_summer_2025.tif"
ndwi_path = output_dir / "ndwi_summer_2025.tif"

city_ndvi = zonal_mean(ndvi_path, nyc_boundary)[0]["mean"]
city_ndwi = zonal_mean(ndwi_path, nyc_boundary)[0]["mean"]

print("\nNYC Summary Statistics")
print(f"NYC Mean NDVI (JFK 2025): {city_ndvi:.4f}")
print(f"NYC Mean NDWI (JFK 2025): {city_ndwi:.4f}")